In [1]:
!pip install bertviz

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for bertviz from https://files.pythonhosted.org/packages/66/07/cce3d29605a3011d3685b2041fb94fcad25565b80bd2f22f3dcd75b2eee9/bertviz-1.4.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/157.6 kB ? eta -:--:--
   ----- --------------------------------- 20.5/157.6 kB 640.0 kB/s eta 0:00:01
   -------------------- ------------------ 81.9/157.6 kB 919.0 kB/s eta 0:00:01
   ---------------------------------------- 157.6/157.6 kB 1.6 MB/s eta 0:00:00


In [3]:
from bertviz import head_view, model_view
from transformers import BertTokenizer, BertModel

In [4]:
model_version = 'bert-base-uncased'
do_lower_case = True
model = BertModel.from_pretrained(model_version, 
                                  output_attentions=True)
tokenizer = BertTokenizer.from_pretrained(model_version,
                                          do_lower_case=do_lower_case)



C:\Users\COMP\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
sentence_a = "A lot of people like animals so they adopt cats"
sentence_b = "A lot of people like animals so they adopt dogs"
inputs = tokenizer.encode_plus(sentence_a, sentence_b, return_tensors='pt', add_special_tokens=True)
token_type_ids = inputs['token_type_ids']
input_ids = inputs['input_ids']
attention = model(input_ids, token_type_ids=token_type_ids)[-1]
sentence_b_start = token_type_ids[0].tolist().index(1)
input_id_list = input_ids[0].tolist() # Batch index 0
tokens = tokenizer.convert_ids_to_tokens(input_id_list)

In [7]:
head_view(attention, tokens)

<IPython.core.display.Javascript object>

In [8]:
%%capture
input_text = "The output shows the attention values" #@param {type:"string"}
print(input_text)

In [9]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name, 
                                  output_attentions=True)
tokens = tokenizer.tokenize(input_text)
input_ids = tokenizer.convert_tokens_to_ids(tokens)

C:\Users\COMP\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
#attention matrix

In [11]:
inputs = tokenizer.encode_plus(input_text, return_tensors='pt')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
outputs = model(input_ids, attention_mask=attention_mask)
attentions = outputs.attentions

In [12]:
#visualizing the attention head activity 

In [13]:
for layer, attention in enumerate(attentions):
    print(f"Layer {layer+1}:")
    for head, head_attention in enumerate(attention[0]):
        print(f"Head {head+1}:")
        for source_token, target_tokens in enumerate(head_attention[:len(tokens)]):
            print(f"Source token '{tokens[source_token]}' (index {source_token+1}):")
            for target_token, attention_value in enumerate(target_tokens[:len(tokens)]):
                print(f"Target token '{tokens[target_token]}' (index {target_token+1}): {attention_value}")

Layer 1:
Head 1:
Source token 'the' (index 1):
Target token 'the' (index 1): 0.070714071393013
Target token 'output' (index 2): 0.1605578511953354
Target token 'shows' (index 3): 0.04948228597640991
Target token 'the' (index 4): 0.05066874995827675
Target token 'attention' (index 5): 0.1661549210548401
Target token 'values' (index 6): 0.08814767003059387
Source token 'output' (index 2):
Target token 'the' (index 1): 0.14874929189682007
Target token 'output' (index 2): 0.15917329490184784
Target token 'shows' (index 3): 0.08759882301092148
Target token 'the' (index 4): 0.08798352628946304
Target token 'attention' (index 5): 0.2005365937948227
Target token 'values' (index 6): 0.06033432111144066
Source token 'shows' (index 3):
Target token 'the' (index 1): 0.08729220926761627
Target token 'output' (index 2): 0.048360973596572876
Target token 'shows' (index 3): 0.18882279098033905
Target token 'the' (index 4): 0.13009601831436157
Target token 'attention' (index 5): 0.05854301527142525
Tar

In [14]:
#Visualizing word relationships using attention scores with pandas

In [15]:
import pandas as pd
import ipywidgets as widgets

In [16]:
df_layers_heads = []
for layer, attention in enumerate(attentions):
    for head, head_attention in enumerate(attention[0]):
        attention_matrix = head_attention[:len(tokens),
                                          :len(tokens)].detach().numpy()
        df_attention = pd.DataFrame(attention_matrix, index=tokens,
                                    columns= tokens)
        df_layers_heads.append((layer, head, df_attention))

In [17]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

def display_attention(selected_layer, selected_head):
    _, _, df_to_display = next(df for df in df_layers_heads if df[0] == selected_layer and df[1] == selected_head)
    display(df_to_display)

In [18]:
layer_widget = widgets.IntSlider(min=0, max=len(attentions)-1, step=1, description='Layer:')
head_widget = widgets.IntSlider(min=0, max=len(attentions[0][0])-1, step=1, description='Head:')

In [19]:
widgets.interact(display_attention, selected_layer=layer_widget, selected_head=head_widget)

interactive(children=(IntSlider(value=0, description='Layer:', max=11), IntSlider(value=0, description='Head:'…

<function __main__.display_attention(selected_layer, selected_head)>

In [20]:
import itertools
# Define the base scores of the words
words = {'I': 0.2, 'love': 0.6, 'playing': 0.5, 'chess': 0.4, 'with': 0.1, 'my': 0.3, 'friends': 0.4}
# Define the bonus scores for certain combinations of words
bonus = {('I', 'love'): 0.3, ('love', 'playing'): 0.25, ('playing', 'chess'): 0.45, ('with', 'my', 'friends'): 0.35}

In [21]:
def total_score(coalition):
    score = sum(words[word] for word in coalition)
    for b in bonus.keys():
        if all(word in coalition for word in b):
            score += bonus[b]
    return score

In [23]:
def shapley_value(word):
    N = len(words)
    permutations = list(itertools.permutations(words))
    marginal_contributions = []
    counter = 0  # Counter initialization
    for permutation in permutations:
        index = permutation.index(word)
        coalition_without_word = permutation[:index]
        coalition_with_word = permutation[:index+1]
        marginal_contribution = total_score(coalition_with_word) - total_score(coalition_without_word)
        marginal_contributions.append(marginal_contribution)
        counter += 1  # Increment counter
    print(f"Processed {counter} permutations")  # Print counter
    return sum(marginal_contributions)

# Calculate the Shapley value of each word
for word in words:
    print(f"The Shapley value of '{word}' is {shapley_value(word)}")
     

Processed 5040 permutations
The Shapley value of 'I' is 1764.0000000000798
Processed 5040 permutations
The Shapley value of 'love' is 4409.99999999998
Processed 5040 permutations
The Shapley value of 'playing' is 4283.999999999768
Processed 5040 permutations
The Shapley value of 'chess' is 3150.000000000016
Processed 5040 permutations
The Shapley value of 'with' is 1092.000000000074
Processed 5040 permutations
The Shapley value of 'my' is 2099.999999999982
Processed 5040 permutations
The Shapley value of 'friends' is 2604.0000000001683


In [24]:
!pip install xformers
!pip install shap

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for xformers from https://files.pythonhosted.org/packages/19/75/9f341504f0342132c4df68083287de58797f932526457c85574a63e5bbbd/xformers-0.0.27-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torch==2.3.1 from https://files.pythonhosted.org/packages/d3/1d/a257913c89572de61316461db91867f87519146e58132cdeace3d9ffbe1f/torch-2.3.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for mkl<=2021.4.0,>=2021.1.1 from https://files.pythonhosted.org/packages/fe/1c/5f6dbf18e8b73e0a5472466f0ea8d48ce9efae39bd2ff38cebf8dce61259/mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata
  Obtaining dependency information for intel-openmp==2021.* from https://files.pythonhosted.org/packages/6f/21/b590c0cc3888b24f2ac9898c41d852d7454a1695fbad34bee85dba6dc408/intel_openmp-2021.4.0-py2.py3-none-win_amd64.whl.metadata
  Obtaining dependency information for tbb==2

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\COMP\\AppData\\Roaming\\Python\\Python311\\site-packages\\~-fuser\\_C.cp311-win_amd64.pyd'
Check the permissions.



Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for shap from https://files.pythonhosted.org/packages/82/29/923869e92c74bf07ec2b9a52ad5ac67d4184c873ba33ada7d4584356463a/shap-0.46.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for slicer==0.0.8 from https://files.pythonhosted.org/packages/63/81/9ef641ff4e12cbcca30e54e72fb0951a2ba195d0cda0ba4100e532d929db/slicer-0.0.8-py3-none-any.whl.metadata
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/d8/ec/ebef2f7d7c28503f958f0f8b992e7ce606fb74f9e891199329d5f5f87404/numpy-1.24.4-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/456.1 kB ? eta -:--:--
   ------------- -------------------------- 153.6/456.1 kB 3.1 MB/s eta 0:00:01
   --------------------------- ------------ 317.4/456.1 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 456.1/456.1 kB 3.6 MB/s eta 0:00:00

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\COMP\\AppData\\Roaming\\Python\\Python311\\site-packages\\~umpy.libs\\libopenblas64__v0.3.23-293-gc2f4bdbb-gcc_10_3_0-2bde3a66a51006b2b53eb373ff767a3f.dll'
Check the permissions.



In [25]:
sentence = 'SHAP is a useful explainer' #@param {type:"string"}

In [26]:

import transformers

# load a transformers pipeline model
model = transformers.pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# analyze the sentiment of the input sentence
result = model(sentence)[0]
print(result)

C:\Users\COMP\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

C:\Users\COMP\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\COMP\.cache\huggingface\hub\models--distilbert-base-uncased-finetuned-sst-2-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

{'label': 'POSITIVE', 'score': 0.9869391918182373}


In [28]:
pip install shap

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for shap from https://files.pythonhosted.org/packages/82/29/923869e92c74bf07ec2b9a52ad5ac67d4184c873ba33ada7d4584356463a/shap-0.46.0-cp311-cp311-win_amd64.whl.metadata
  Using cached shap-0.46.0-cp311-cp311-win_amd64.whl.metadata (25 kB)
Using cached shap-0.46.0-cp311-cp311-win_amd64.whl (456 kB)
Note: you may need to restart the kernel to use updated packages.
